In [1]:
from matplotlib import pyplot
from matplotlib.image import imread
from pandas import read_csv
import os


import numpy as np
import pandas as pd
pd.options.display.max_rows
import os, sys
import cv2

from keras.models import Sequential
from keras.layers import Dense, Flatten, Activation, Dropout, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras import layers

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from keras import optimizers
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt


# load and prepare planet dataset and save to file
from os import listdir
from numpy import zeros
from numpy import asarray
from numpy import savez_compressed
from pandas import read_csv
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

Using TensorFlow backend.


# prepare the data for modeling

takes csv file with photo_name|tag_names, 
separates the space delimited tags by commas,
adds tags to a unique set of all tags
converts that set into a list
orders the list alphabetically
instantiate a dict that maps labels : integers
and another for integers : labels

In [4]:
# createa  mapping of tags to integers using a dict

def create_tag_mapping(mapping_csv):
    # create a set of all known tags
    labels = set()
    for i in range(len(mapping_csv)):
        # convert spaced separated tags into an array of tags
        tags = mapping_csv['attribute_ids'][i].split(' ')
        # add tags to the set of known labels
        labels.update(tags)
    # convert set of labels to a list to list
    labels = list(labels)
    # order set alphabetically
    labels.sort()
    # dict that maps labels to integers, and the reverse
    labels_map = {labels[i]:i for i in range(len(labels))}
    inv_labels_map = {i:labels[i] for i in range(len(labels))}
    return labels_map, inv_labels_map       

In [7]:
create_tag_mapping(read_csv('train.csv'))

({'0': 0,
  '1': 1,
  '10': 2,
  '100': 3,
  '1000': 4,
  '1001': 5,
  '1002': 6,
  '1003': 7,
  '1004': 8,
  '1005': 9,
  '1006': 10,
  '1007': 11,
  '1008': 12,
  '1009': 13,
  '101': 14,
  '1010': 15,
  '1011': 16,
  '1012': 17,
  '1013': 18,
  '1014': 19,
  '1015': 20,
  '1016': 21,
  '1017': 22,
  '1018': 23,
  '1019': 24,
  '102': 25,
  '1020': 26,
  '1021': 27,
  '1022': 28,
  '1023': 29,
  '1024': 30,
  '1025': 31,
  '1026': 32,
  '1027': 33,
  '1028': 34,
  '1029': 35,
  '103': 36,
  '1030': 37,
  '1031': 38,
  '1032': 39,
  '1033': 40,
  '1034': 41,
  '1035': 42,
  '1036': 43,
  '1037': 44,
  '1038': 45,
  '1039': 46,
  '104': 47,
  '1040': 48,
  '1041': 49,
  '1042': 50,
  '1043': 51,
  '1044': 52,
  '1045': 53,
  '1046': 54,
  '1047': 55,
  '1048': 56,
  '1049': 57,
  '105': 58,
  '1050': 59,
  '1051': 60,
  '1052': 61,
  '1053': 62,
  '1054': 63,
  '1055': 64,
  '1056': 65,
  '1057': 66,
  '1058': 67,
  '1059': 68,
  '106': 69,
  '1060': 70,
  '1061': 71,
  '1062': 72,
  '

In [ ]:
# create list of photo names from the train photo repo to compare against the train csv and make sure all 
 # photos are accounted for

photo_names_list = []
for file in os.listdir('train'):
    photo_names_list.append(file.rstrip('.png'))
    
photo_list = pd.DataFrame(photo_names_list)
photo_list.to_csv('photo_names.csv')
photo_list.head()

In [11]:
# create a mapping of filename to a list of tags
def create_file_mapping(mapping_csv):
    mapping = dict()
    for i in range(len(mapping_csv)):
        name, tags = mapping_csv['id'][i], mapping_csv['attribute_ids'][i]
        mapping[name] = tags.split(' ')
    return mapping

In [12]:
create_file_mapping(read_csv('train.csv'))

{'1000483014d91860': ['147', '616', '813'],
 '1000fe2e667721fe': ['51', '616', '734', '813'],
 '1001614cb89646ee': ['776'],
 '10041eb49b297c08': ['51', '671', '698', '813', '1092'],
 '100501c227f8beea': ['13', '404', '492', '903', '1093'],
 '10050ed12fbad46d': ['189', '279', '774', '800', '1051'],
 '100543a032517972': ['188', '1034'],
 '1006665c0aad488': ['1010', '1053'],
 '1007057734dba6df': ['189', '541', '542', '993'],
 '1008abd71f3ed5bc': ['70', '776', '794', '813', '1046', '1092'],
 '1008c7837081f985': ['79', '1062'],
 '1009f5737fc77f2': ['188', '668', '754'],
 '100a0dcde728cb36': ['51', '675'],
 '100a58282c6584bf': ['147', '716', '903', '1092'],
 '100b45b7c4020f5d': ['161', '489', '704', '1100'],
 '100bb499d37d0751': ['188', '535'],
 '100bbf5e832083d3': ['51', '212', '426', '586', '940'],
 '100d750286e85bf3': ['13', '616'],
 '100e1e65a6d7850e': ['737', '1009', '1046'],
 '100ef61c00e1b5d3': ['25', '161', '784', '1059'],
 '100efeead4d0f90c': ['189', '542', '670', '813', '949', '109

In [13]:
# create a one hot encoding for one list of tags
def one_hot_encode(tags, mapping):
    # create empty vector
    encoding = zeros(len(mapping), dtype='uint8')
    # mark 1 for each tag in the vector
    for tag in tags:
        encoding[mapping[tag]] = 1
    return encoding

In [14]:
# load all images into memory
def load_dataset(path, file_mapping, tag_mapping):
    photos, targets = list(), list()
    # enumerate files in the directory
    for filename in listdir(folder):
        # load image
        photo = load_img(path + filename, target_size=(128,128))
        # convert to numpy array
        photo = img_to_array(photo, dtype='uint8')
        # get tags
        tags = file_mapping[filename[:-4]]
        # one hot encode tags
        target = one_hot_encode(tags, tag_mapping)
        # store
        photos.append(photo)
        targets.append(target)
    X = asarray(photos, dtype='uint8')
    y = asarray(targets, dtype='uint8')
    return X, y

In [15]:
# load the mapping file
filename = 'train.csv'
mapping_csv = read_csv(filename)
# create a mapping of tags to integers
tag_mapping, _ = create_tag_mapping(mapping_csv)
# create a mapping of filenames to tag lists
file_mapping = create_file_mapping(mapping_csv)
# load the jpeg images
folder = 'train/'
X, y = load_dataset(folder, file_mapping, tag_mapping)
print(X.shape, y.shape)
# save both arrays to one file in compressed format
savez_compressed('iMet_data.npz', X, y)

(109237, 128, 128, 3) (109237, 1103)


# Model Evaluation Measure

In [17]:
# compare f-beta score between sklearn and keras
from numpy import load
from numpy import ones
from numpy import asarray
from sklearn.model_selection import train_test_split
from sklearn.metrics import fbeta_score
from keras import backend

# load train and test dataset
def load_dataset():
	# load dataset
	data = load('iMet_data.npz')
	X, y = data['arr_0'], data['arr_1']
	# separate into train and test datasets
	trainX, testX, trainY, testY = train_test_split(X, y, test_size=0.3, random_state=1)
	print(trainX.shape, trainY.shape, testX.shape, testY.shape)
	return trainX, trainY, testX, testY

# calculate fbeta score for multi-class/label classification
def fbeta(y_true, y_pred, beta=2):
	# clip predictions
	y_pred = backend.clip(y_pred, 0, 1)
	# calculate elements
	tp = backend.sum(backend.round(backend.clip(y_true * y_pred, 0, 1)), axis=1)
	fp = backend.sum(backend.round(backend.clip(y_pred - y_true, 0, 1)), axis=1)
	fn = backend.sum(backend.round(backend.clip(y_true - y_pred, 0, 1)), axis=1)
	# calculate precision
	p = tp / (tp + fp + backend.epsilon())
	# calculate recall
	r = tp / (tp + fn + backend.epsilon())
	# calculate fbeta, averaged across each class
	bb = beta ** 2
	fbeta_score = backend.mean((1 + bb) * (p * r) / (bb * p + r + backend.epsilon()))
	return fbeta_score

# load dataset
trainX, trainY, testX, testY = load_dataset()
# make all one predictions
train_yhat = asarray([ones(trainY.shape[1]) for _ in range(trainY.shape[0])])
test_yhat = asarray([ones(testY.shape[1]) for _ in range(testY.shape[0])])
# evaluate predictions with sklearn
train_score = fbeta_score(trainY, train_yhat, 2, average='samples')
test_score = fbeta_score(testY, test_yhat, 2, average='samples')
print('All Ones (sklearn): train=%.3f, test=%.3f' % (train_score, test_score))
# evaluate predictions with keras
train_score = fbeta(backend.variable(trainY), backend.variable(train_yhat)).eval(session=backend.get_session())
test_score = fbeta(backend.variable(testY), backend.variable(test_yhat)).eval(session=backend.get_session())
print('All Ones (keras): train=%.3f, test=%.3f' % (train_score, test_score))

(76465, 128, 128, 3) (76465, 1103) (32772, 128, 128, 3) (32772, 1103)
All Ones (sklearn): train=0.014, test=0.014
All Ones (keras): train=0.014, test=0.014


# Evaluate a Baseline Model

In [20]:
# baseline model for the planet dataset
import sys
from numpy import load
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from keras import backend
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD

# load train and test dataset
def load_dataset():
	# load dataset
	data = load('iMet_data.npz')
	X, y = data['arr_0'], data['arr_1']
	# separate into train and test datasets
	trainX, testX, trainY, testY = train_test_split(X, y, test_size=0.3, random_state=1)
	print(trainX.shape, trainY.shape, testX.shape, testY.shape)
	return trainX, trainY, testX, testY

# calculate fbeta score for multi-class/label classification
def fbeta(y_true, y_pred, beta=2):
	# clip predictions
	y_pred = backend.clip(y_pred, 0, 1)
	# calculate elements
	tp = backend.sum(backend.round(backend.clip(y_true * y_pred, 0, 1)), axis=1)
	fp = backend.sum(backend.round(backend.clip(y_pred - y_true, 0, 1)), axis=1)
	fn = backend.sum(backend.round(backend.clip(y_true - y_pred, 0, 1)), axis=1)
	# calculate precision
	p = tp / (tp + fp + backend.epsilon())
	# calculate recall
	r = tp / (tp + fn + backend.epsilon())
	# calculate fbeta, averaged across each class
	bb = beta ** 2
	fbeta_score = backend.mean((1 + bb) * (p * r) / (bb * p + r + backend.epsilon()))
	return fbeta_score

# define cnn model
def define_model(in_shape=(128, 128, 3), out_shape=1103):
	model = Sequential()
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=in_shape))
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Flatten())
	model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dense(out_shape, activation='sigmoid'))
	# compile model
	opt = SGD(lr=0.01, momentum=0.9)
	model.compile(optimizer=opt, loss='binary_crossentropy', metrics=[fbeta])
	return model

# plot diagnostic learning curves
def summarize_diagnostics(history):
	# plot loss
	pyplot.subplot(211)
	pyplot.title('Cross Entropy Loss')
	pyplot.plot(history.history['loss'], color='blue', label='train')
	pyplot.plot(history.history['val_loss'], color='orange', label='test')
	# plot accuracy
	pyplot.subplot(212)
	pyplot.title('Fbeta')
	pyplot.plot(history.history['fbeta'], color='blue', label='train')
	pyplot.plot(history.history['val_fbeta'], color='orange', label='test')
	# save plot to file
	filename = sys.argv[0].split('/')[-1]
	pyplot.savefig(filename + '_plot.png')
	pyplot.close()

# run the test harness for evaluating a model
def run_test_harness():
	# load dataset
	trainX, trainY, testX, testY = load_dataset()
	# create data generator
	datagen = ImageDataGenerator(rescale=1.0/255.0)
	# prepare iterators
	train_it = datagen.flow(trainX, trainY, batch_size=128)
	test_it = datagen.flow(testX, testY, batch_size=128)
	# define model
	model = define_model()
	# fit model
	history = model.fit_generator(train_it, steps_per_epoch=len(train_it),
		validation_data=test_it, validation_steps=len(test_it), epochs=50, verbose=0)
	# evaluate model
	loss, fbeta = model.evaluate_generator(test_it, steps=len(test_it), verbose=0)
	print('> loss=%.3f, fbeta=%.3f' % (loss, fbeta))
	# learning curves
	summarize_diagnostics(history)

# entry point, run the test harness
run_test_harness()

MemoryError: 